In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
EXPERIMENT_NAME = 'P3856'

In [3]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 2
RT_TOLERANCE = 1

In [4]:
# load the detected features
# EXPERIMENT_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/{}'.format(EXPERIMENT_NAME)
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_FILE_NAME = '{}/recalibrated-features/experiment-features.pkl'.format(EXPERIMENT_DIR)

In [5]:
MAPPING_FILE = '{}/recalibrated-percolator-output/percolator-idx-mapping.csv'.format(EXPERIMENT_DIR)
SELECTED_RUN = 'P3856_YHE211_1_Slot1-1_1_5104'
map_df = pd.read_csv(MAPPING_FILE, sep=',')
perc_id = map_df[(map_df.run_name == SELECTED_RUN)].iloc[0].file_idx

In [6]:
features_df = pd.read_pickle(FEATURES_FILE_NAME)
features_df = features_df[(features_df.percolator_idx == perc_id)]

In [7]:
features_df.columns

Index(['feature_id', 'charge', 'rt_apex', 'rt_peak_width', 'scan_apex',
       'scan_peak_width', 'intensity', 'precursor_id', 'monoisotopic_mass',
       'predicted_mass_error', 'recalibrated_monoisotopic_mass',
       'recalibrated_monoisotopic_mz', 'percolator_idx'],
      dtype='object')

In [8]:
features_df['mz'] = features_df.recalibrated_monoisotopic_mz  # shorthand to reduce verbosity
features_df['mz_ppm_tolerance'] = features_df.mz * MZ_TOLERANCE_PERCENT / 100
features_df['mz_lower'] = features_df.mz - features_df.mz_ppm_tolerance
features_df['mz_upper'] = features_df.mz + features_df.mz_ppm_tolerance
features_df['scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['rt_upper'] = features_df.rt_apex + RT_TOLERANCE
features_df['composite_key'] = features_df.apply(lambda row: '{},{},{}'.format(row.feature_id, row.precursor_id, row.percolator_idx), axis=1)


In [13]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('mz')
columns_to_drop_l.append('mz_ppm_tolerance')
columns_to_drop_l.append('mz_lower')
columns_to_drop_l.append('mz_upper')
columns_to_drop_l.append('scan_lower')
columns_to_drop_l.append('scan_upper')
columns_to_drop_l.append('rt_lower')
columns_to_drop_l.append('rt_upper')
columns_to_drop_l.append('composite_key')


In [14]:
features_df.sort_values(by=['intensity'], ascending=False, inplace=True)

In [15]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.mz > row.mz_lower) & (features_df.mz < row.mz_upper) & (features_df.scan_apex > row.scan_lower) & (features_df.scan_apex < row.scan_upper) & (features_df.rt_apex > row.rt_lower) & (features_df.rt_apex < row.rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].composite_key)


In [16]:
# remove any extractions that are not in the keep list
dedup_df = features_df[features_df.composite_key.isin(keep_l)].copy()

In [26]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 23544 duplicates (10% of the original detections)
there are 211333 detected de-duplicated features


In [20]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [21]:
dedup_df.sample(n=3)

,feature_id,charge,rt_apex,rt_peak_width,scan_apex,scan_peak_width,intensity,precursor_id,monoisotopic_mass,predicted_mass_error,recalibrated_monoisotopic_mass,recalibrated_monoisotopic_mz,percolator_idx
2,4339703,2,2192.04,20.00,302.04,75.00,12652.309906,43397,1993.662062,0.002621,1993.659441,997.836996,29
4,4616905,1,2236.12,20.00,432.58,75.00,2983.000000,46169,848.831008,-0.000848,848.831855,849.839131,29
0,2289301,2,1879.27,5.97,220.57,48.73,224693.000000,22893,2087.988752,-0.000600,2087.989352,1045.001952,29


In [27]:
dedup_df.to_pickle('/home/ubuntu/P3856_YHE211_1_Slot1-1_1_5104-dedup-features.pkl')